In [2]:
import os
import numpy as np
import nibabel as nib

In [ ]:
# 入力ディレクトリ（任意で書き換えてください）
input_dir = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/S_train"

# カットファイルをパースする関数
def parse_cut_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    cut_info = []
    for line in lines:
        # 末尾の不要なカンマを除去
        line = line.strip().rstrip(',')
        values = list(map(int, line.split(',')))

        vertebra_num = values[0]
        slb, slb2 = values[1:3]
        slice_count = values[3]
        base_size = values[4]
        x_range = values[5:7]
        y_range = values[7:9]
        z_range = values[9:11]
        cut_info.append({
            "vertebra_num": vertebra_num,
            "slb": slb,
            "slb2": slb2,
            "slice_count": slice_count,
            "base_size": base_size,
            "x_range": x_range,
            "y_range": y_range,
            "z_range": z_range
        })
    return cut_info

# NIfTIファイルをカット＋骨領域抽出する関数
def apply_cut_to_nifti(input_path, output_path, cut_info):
    img_nifti = nib.load(input_path)
    img = img_nifti.get_fdata()

    # 座標範囲に基づいてカット
    x_start, x_end = cut_info["x_range"]
    y_start, y_end = cut_info["y_range"]
    z_start, z_end = cut_info["z_range"]

    # 外側に少しだけ余裕を持たせる（±1）
    x_start -= 1
    x_end   += 1
    y_start -= 1
    y_end   += 1
    z_start -= 1
    z_end   += 1

    # 範囲外を修正
    x_end = min(x_end, img.shape[0])
    y_end = min(y_end, img.shape[1])
    z_end = min(z_end, img.shape[2])

    # カット後の画像データ
    cut_data = img[x_start:x_end, y_start:y_end, z_start:z_end]

    # 骨領域のみ残す（背景を0）この処理いらない
    #bone_min = 100
    #bone_max = 2000
    #bone_mask = (cut_data >= bone_min) & (cut_data <= bone_max)
    #cut_data = np.where(bone_mask, cut_data, 0)

    # 保存用NIfTI画像を作成
    cut_img_nifti = nib.Nifti1Image(cut_data, img_nifti.affine, img_nifti.header)
    nib.save(cut_img_nifti, output_path)

# ディレクトリ内のファイルを処理する関数（1003番かつ椎体番号27のみ出力）
def process_directory(dir_path):
    # （例）cut_li1003.txt のようなファイルを探す
    files = os.listdir(dir_path)
    cut_files = [f for f in files if f.startswith("cut_li") and f.endswith(".txt")]
    nii_files = [f for f in files if f.endswith(".nii") or f.endswith(".nii.gz")]

    # 「1003」という数字が入ったカットファイルだけを対象にする
    cut_files_1003 = [f for f in cut_files if "1003" in f]
    # 同様に「inp1003」というNIfTIファイルがあると仮定
    nii_files_1003 = [f for f in nii_files if "inp1003" in f]

    # カットファイルがない場合は何もせず終了
    if not cut_files_1003 or not nii_files_1003:
        print("対象ファイルが見つかりませんでした。")
        return

    # 例: 「cut_li1003.txt」が1つだけ存在すると想定
    cut_info_list = parse_cut_file(os.path.join(dir_path, cut_files_1003[0]))

    # カット情報の中から椎体番号 27 のものだけ処理
    target_cut_info = [info for info in cut_info_list if info["vertebra_num"] == 27]

    if not target_cut_info:
        print("椎体番号 27 のカット情報が見つかりませんでした。")
        return

    # inp1003 のNIfTIファイルも1つだけあると想定
    input_nifti_path = os.path.join(dir_path, nii_files_1003[0])

    # 出力先を sample_train/inp1003/27/cut_inp1003.nii.gz とする
    # ディレクトリを作成
    out_dir = os.path.join(dir_path, "inp1003", "27")
    os.makedirs(out_dir, exist_ok=True)
    output_path = os.path.join(out_dir, "cut_inp1003_test_1.nii.gz")

    # 椎体番号27のカット情報は複数行あっても1つとみなして処理する場合
    # （複数行あるなら最初の情報でカット処理という想定）
    apply_cut_to_nifti(input_nifti_path, output_path, target_cut_info[0])

    print("出力が完了しました。:", output_path)

# 実行
process_directory(input_dir)

出力が完了しました。: /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/S_train/inp1003/27/cut_inp1003_test_1.nii.gz


In [1]:
import os
import re
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import nibabel as nib

In [2]:
"""
高速版 NIfTI カットスクリプト

改善点
* nibabel の遅延読み込み (mmap) で必要領域のみをロード
* ProcessPoolExecutor によるマルチプロセス並列化
* ファイルリストを辞書キャッシュして線形探索を 1 回に集約
* dtype を保持して余分な float64 変換を回避
"""
# パース関連
# -----------------------------

def parse_cut_file(file_path: str):
    """cut_li*.txt を辞書のリストに変換"""
    cut_info = []
    with open(file_path) as f:
        for line in f:
            line = line.strip().rstrip(',')
            if not line:
                continue
            vals = list(map(int, line.split(',')))
            cut_info.append({
                "vertebra_num": vals[0],
                "slb": vals[1],
                "slb2": vals[2],
                "slice_count": vals[3],
                "base_size": vals[4],
                "x_range": vals[5:7],
                "y_range": vals[7:9],
                "z_range": vals[9:11]
            })
    return cut_info

# -----------------------------
# NIfTI 切り出し
# -----------------------------

def _clip(start: int, end: int, maxv: int, margin: int = 1):
    """範囲を margin 分だけ広げてクリップ"""
    return max(start - margin, 0), min(end + margin, maxv)

def apply_cut_to_nifti(args):
    """ワーカー関数: 指定領域を切り出して保存"""
    input_path, output_path, cut_info = args
    img = nib.load(input_path, mmap=True)  # lazy + memmap

    xs, xe = _clip(*cut_info["x_range"], img.shape[0])
    ys, ye = _clip(*cut_info["y_range"], img.shape[1])
    zs, ze = _clip(*cut_info["z_range"], img.shape[2])

    # 必要領域だけを実体化
    cut_data = np.asanyarray(img.dataobj[xs:xe, ys:ye, zs:ze])

    nib.save(nib.Nifti1Image(cut_data, img.affine, img.header), output_path)
    return output_path

# -----------------------------
# ディレクトリ処理
# -----------------------------

def process_directory(input_dir: str, output_base_dir: str, max_workers: int | None = None):
    files = os.listdir(input_dir)
    cut_files = [f for f in files if f.startswith("cut_li") and f.endswith(".txt")]
    nii_files = [f for f in files if f.endswith(".nii") or f.endswith(".nii.gz")]

    # 数字 -> 対応する nii ファイル群
    file_map: dict[str, list[str]] = defaultdict(list)
    for f in nii_files:
        num = re.search(r"\d+", f).group()
        file_map[num].append(f)

    # 並列実行用ジョブ作成
    jobs: list[tuple[str, str, dict]] = []
    for cut_file in cut_files:
        number = re.search(r"\d+", cut_file).group()
        cut_info_list = parse_cut_file(os.path.join(input_dir, cut_file))
        for cut_info in cut_info_list:
            for nii_file in file_map[number]:
                in_path = os.path.join(input_dir, nii_file)
                out_dir = os.path.join(output_base_dir, f"inp{number}", str(cut_info["vertebra_num"]))
                os.makedirs(out_dir, exist_ok=True)
                out_path = os.path.join(out_dir, f"cut_{nii_file}")
                jobs.append((in_path, out_path, cut_info))

    # マルチプロセス実行
    with ProcessPoolExecutor(max_workers=max_workers or os.cpu_count()) as ex:
        futures = [ex.submit(apply_cut_to_nifti, j) for j in jobs]
        for _ in as_completed(futures):
            pass  # 進捗だけ待つ

# -----------------------------
# エントリポイント
# -----------------------------

if __name__ == "__main__":
    train_dir = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/S_train"
    # val_dir   = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/S_val"
    # test_dir  = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/S_test"

    output_train_dir = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/processed_train"
    # output_val_dir   = "..."  # 必要なら指定

    process_directory(train_dir, output_train_dir)
    # process_directory(val_dir, output_val_dir)
    # process_directory(test_dir, output_test_dir)

    print("カット処理が完了しました。")


カット処理が完了しました。
